# GPU Programming Concepts

Welcome to the webinar _GPU Programming Concepts_. In this webinar you will learn several techniques for scaling single GPU applications to multi-GPU and multiple nodes, with an emphasis on [NCCL (NVIDIA Collective Communications Library)](https://docs.nvidia.com/deeplearning/sdk/nccl-developer-guide/docs/index.html), [CUDA-aware MPI](https://developer.nvidia.com/blog/introduction-cuda-aware-mpi/), and [NVSHMEM](https://developer.nvidia.com/nvshmem) which allows for elegant multi-GPU application code and has been proven to scale very well on systems with many GPUs.

## The Coding Environment

The first step is display information about the CPU architecture with the command `lscpu`

In [ ]:
!lscpu

In this node, we can observe that the multi-GPU resources connect with the NUMA nodes.

For your work today, you have access to several GPUs in the cloud. Run the following cell to see the GPUs available to you today.

In [ ]:
!nvidia-smi topo -m 

While your work today will be on a single node, all the techniques you learn today, in particular CUDAWARE-MPI and NVSHMEM, can be used to run your applications across clusters of multi-GPU nodes.

Let us show the NVLink Status for different GPUs reported from `nvidia-smi`:

In [ ]:
!nvidia-smi nvlink --status -i 0

In the end, it gives information about the NUMA memory nodes, with tue `lstopo` command, that is used to show the topology of the system.  

In [ ]:
!lstopo --of png > ogbon.png

This will import and display a .png image in Jupyter:

In [ ]:
from IPython.display import display
from PIL import Image
path="ogbon.png"
display(Image.open(path))

## Table of Contents

During this short course today you will work through each of the following notebooks:

- [_NCCL_](2-OGBON-NCCL-P2P.ipynb): In this notebook you will introduced the NCCL API, and the concepts of peer-to-peer communication between GPUs.

- [_CUDA-aware MPI_](3-OGBON-CUDAWARE-MPI.ipynb): You will begin by familiarizing with the concepts of CUDA-aware MPI API to multi-GPU nodes.

- [_Monte Carlo Approximation of $\pi$ - Single GPU_](4-OGBON-MCπ-SGPU.ipynb): You will begin by familiarizing yourself with a single GPU implementation of the monte-carlo approximation of π algorithm, which we will use to introduce many multi GPU programming paradigms.

- [_Monte Carlo Approximation of $\pi$ - Multiple GPUs_](5-OGBON-MCπ-MGPU.ipynb): In this notebook you will extend the monte-carlo π program to run on multiple GPUs by looping over available GPU devices.

- [_Monte Carlo Approximation of $\pi$ - CUDA-aware MPI_](6-OGBON-MCπ-CUDAWARE-MPI): In this notebook you will learn how to applied the CUDA-aware MPI, and some concepts about peer-to-peer communication between GPUs in the SPMD paradigm.

- [_Monte Carlo Approximation of $\pi$ - NVSHMEM_](7-OGBON-MCπ-NVSHMEM.ipynb): In this notebook you will be introduced to NVSHMEM, and will take your first pass with it using the monte-carlo π program.

- [_Jacobi Iteration_](8-OGBON-Jacobi.ipynb): In this notebook you will be introduced to a Laplace equation solver using Jacobi iteration and will learn how to use NVSHMEM to handle boundary communications between multiple GPUs.

## Clear the Memory

Before moving on, please execute the following cell to clear up the CPU memory. This is required to move on to the next notebook.

In [ ]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

## Next

In the next section, you will see  how to learn the GPU concepts using the API NCCL [_2-OGBON-NCCL-P2P.ipynb_](2-OGBON-NCCL-P2P.ipynb)